### LangGraph使用该工具搜索

In [ ]:
from app.utils.search import SearchEngineFactory
from app.utils.search import SearchEngineType
from langchain.agents import tool


# 搜索工具函数
@tool
def search_tool_factory(query: str) -> dict:
    """
    使用Tavily搜索引擎执行网络搜索并返回最新、相关的搜索结果。
    
    此工具可用于获取实时资讯、新闻报道和网络内容，特别适合寻找时事动态和最新发展。
    
    参数:
        query (str): 要搜索的查询字符串，支持中英文
        
    返回:
        dict: 包含搜索结果的字典，结构如下:
            {
                "query": "原始查询内容",
                "follow_up_questions": None 或 [建议的后续问题列表],
                "answer": None 或 基于结果的摘要回答,
                "images": [] 或 [相关图片链接列表],
                "results": [
                    {
                        "title": "网页标题",
                        "url": "网页URL",
                        "content": "网页内容摘要",
                        "score": 相关性评分(0-1之间的浮点数),
                        "raw_content": None 或 原始内容
                    },
                    ...
                ],
                "response_time": 响应时间(秒)
            }
            
    搜索结果按相关性排序，通常返回5-10条结果。每条结果包含标题、URL、内容摘要和相关性评分。
    
    示例:
        >>> result = search_tool_factory("特朗普最新关于中美贸易战的态度")
        >>> print(result["results"][0]["title"])
        '特朗普对贸易战孤注一掷，威胁其他关键对华谈判 - 纽约时报中文网'
        >>> print(f"搜索用时: {result['response_time']}秒")
        '搜索用时: 1.76秒'
    """
    search_tool = SearchEngineFactory.create_engine(SearchEngineType.TAVILY)
    search_result = search_tool.search(query)
    return search_result

# search_engine.py

In [ ]:
from abc import ABC, abstractmethod
from typing import List, Dict, Any
import asyncio
from tavily import TavilyClient
from dotenv import load_dotenv
import os

# 抽象搜索引擎接口
class SearchEngine(ABC):
    @abstractmethod
    def search(self, query: str) -> Dict[str, Any]:
        """执行搜索查询"""
        pass
    
    @abstractmethod
    def search_with_subqueries(self, subqueries: List[str]) -> List[Dict[str, Any]]:
        """执行子查询搜索"""
        pass
    
    @abstractmethod
    async def search_async(self, queries: List[str], max_concurrency: int = 5) -> List[Dict[str, Any]]:
        """执行异步搜索"""
        pass

# Tavily搜索引擎实现
class TavilySearchEngine(SearchEngine):
    def __init__(self, api_key: str):
        """初始化Tavily搜索引擎
        
        Args:
            api_key: Tavily API密钥
        """
        self.client = TavilyClient(api_key=api_key)
    
    def search(self, query: str) -> Dict[str, Any]:
        """常规搜索方法
        
        Args:
            query: 搜索查询字符串
            
        Returns:
            搜索结果字典
        """
        try:
            response = self.client.search(query)
            return response
        except Exception as e:
            print(f"搜索出错: {e}")
            return {"error": str(e)}
    
    def search_with_subqueries(self, subqueries: List[str]) -> List[Dict[str, Any]]:
        """将查询拆分为较小的子查询进行搜索
        
        Args:
            subqueries: 子查询列表
            
        Returns:
            搜索结果列表
        """
        results = []
        for query in subqueries:
            try:
                result = self.client.search(query)
                results.append(result)
            except Exception as e:
                print(f"子查询 '{query}' 搜索出错: {e}")
                results.append({"query": query, "error": str(e)})
        return results
    
    async def search_async(self, queries: List[str], max_concurrency: int = 5) -> List[Dict[str, Any]]:
        """异步搜索多个查询
        
        Args:
            queries: 查询列表
            max_concurrency: 最大并发请求数
            
        Returns:
            搜索结果列表
        """
        async def _search_one(query: str) -> Dict[str, Any]:
            try:
                loop = asyncio.get_event_loop()
                # 在异步环境中调用同步方法
                result = await loop.run_in_executor(None, lambda: self.client.search(query))
                return result
            except Exception as e:
                print(f"异步查询 '{query}' 搜索出错: {e}")
                return {"query": query, "error": str(e)}
        
        # 限制并发请求数
        semaphore = asyncio.Semaphore(max_concurrency)
        
        async def _search_with_limit(query: str) -> Dict[str, Any]:
            async with semaphore:
                return await _search_one(query)
        
        # 并发执行所有查询
        tasks = [_search_with_limit(query) for query in queries]
        results = await asyncio.gather(*tasks)
        return results

# 可以在此添加其他搜索引擎实现
# class GoogleSearchEngine(SearchEngine):
#     def __init__(self, api_key: str):
#         """初始化Google搜索引擎"""
#         self.api_key = api_key
#         # 初始化Google搜索客户端
#     
#     def search(self, query: str) -> Dict[str, Any]:
#         """实现Google搜索逻辑"""
#         pass
#     
#     def search_with_subqueries(self, subqueries: List[str]) -> List[Dict[str, Any]]:
#         """实现Google子查询搜索逻辑"""
#         pass
#     
#     async def search_async(self, queries: List[str], max_concurrency: int = 5) -> List[Dict[str, Any]]:
#         """实现Google异步搜索逻辑"""
#         pass 

# search_factory.py

In [ ]:
from search_engine import SearchEngine, TavilySearchEngine
from search_engine_type import SearchEngineType
from dotenv import load_dotenv
import os

class SearchEngineFactory:
    """搜索引擎工厂类，负责创建不同类型的搜索引擎实例"""
    
    @staticmethod
    def create_engine(engine_type: SearchEngineType, **kwargs) -> SearchEngine:
        """创建搜索引擎实例
        
        Args:
            engine_type: 搜索引擎类型（枚举）
            **kwargs: 搜索引擎配置参数
            
        Returns:
            搜索引擎实例
        
        Raises:
            ValueError: 当指定的搜索引擎类型不受支持时
        """
        if engine_type == SearchEngineType.TAVILY:
            api_key = kwargs.get("api_key")
            if not api_key:
                load_dotenv()
                api_key = os.getenv("TAVILY_API_KEY")
            return TavilySearchEngine(api_key=api_key)
        # 可以在此添加其他搜索引擎的支持
        # elif engine_type == SearchEngineType.GOOGLE:
        #     return GoogleSearchEngine(**kwargs)
        # elif engine_type == SearchEngineType.BING:
        #     return BingSearchEngine(**kwargs)
        else:
            raise ValueError(f"不支持的搜索引擎类型: {engine_type}") 

# search_engine_type.py

In [ ]:
from enum import Enum, auto

class SearchEngineType(Enum):
    """搜索引擎类型枚举"""
    TAVILY = auto()  # Tavily搜索引擎
    GOOGLE = auto()  # Google搜索引擎
    BING = auto()    # Bing搜索引擎
    # 可以添加更多搜索引擎类型
    
    def __str__(self):
        return self.name.lower() 

# search_service.py

In [ ]:
from typing import Optional, List, Dict, Any
from search_engine import SearchEngine
from search_factory import SearchEngineFactory
from search_engine_type import SearchEngineType

class SearchService:
    """搜索服务单例类，提供统一的搜索接口"""
    
    _instance: Optional["SearchService"] = None
    _engine: Optional[SearchEngine] = None
    
    def __new__(cls, *args, **kwargs):
        """确保SearchService是单例"""
        if cls._instance is None:
            cls._instance = super(SearchService, cls).__new__(cls)
            cls._instance._initialized = False
        return cls._instance
    
    def __init__(self, engine_type: SearchEngineType = SearchEngineType.TAVILY, **kwargs):
        """初始化搜索服务
        
        Args:
            engine_type: 搜索引擎类型（枚举）
            **kwargs: 搜索引擎配置参数
        """
        # 如果已初始化，则不再重复初始化
        if hasattr(self, "_initialized") and self._initialized:
            return
            
        self._engine = SearchEngineFactory.create_engine(engine_type, **kwargs)
        self._initialized = True
    
    def change_engine(self, engine_type: SearchEngineType, **kwargs):
        """更换搜索引擎
        
        Args:
            engine_type: 搜索引擎类型（枚举）
            **kwargs: 搜索引擎配置参数
        """
        self._engine = SearchEngineFactory.create_engine(engine_type, **kwargs)
    
    def search(self, query: str) -> Dict[str, Any]:
        """常规搜索方法
        
        Args:
            query: 搜索查询字符串
            
        Returns:
            搜索结果字典
        """
        return self._engine.search(query)
    
    def search_with_subqueries(self, subqueries: List[str]) -> List[Dict[str, Any]]:
        """将查询拆分为较小的子查询进行搜索
        
        Args:
            subqueries: 子查询列表
            
        Returns:
            搜索结果列表
        """
        return self._engine.search_with_subqueries(subqueries)
    
    async def search_async(self, queries: List[str], max_concurrency: int = 5) -> List[Dict[str, Any]]:
        """异步搜索多个查询
        
        Args:
            queries: 查询列表
            max_concurrency: 最大并发请求数
            
        Returns:
            搜索结果列表
        """
        return await self._engine.search_async(queries, max_concurrency) 